STEP 1 — Setup & load dataset





In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("vehicles.csv")
df.head()
df.shape

(426880, 26)

STEP 2 — Remove duplicates



In [6]:
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Rows before: {before}, after removing duplicates: {after}, removed: {before-after}")

Rows before: 426880, after removing duplicates: 426880, removed: 0


STEP 3 — Inspect columns and basic info


In [7]:
print(df.columns.tolist())
df.info(memory_usage="deep")
df.describe(include='all').T

['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color', 'image_url', 'description', 'county', 'state', 'lat', 'long', 'posting_date']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,426880.0,NaN,NaN,NaN,7311486634.224333,4473170.412559,7207408119.0,7308143339.25,7312620821.0,7315253543.5,7317101084.0
url,426880,426880,https://prescott.craigslist.org/cto/d/prescott...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region,426880,404,columbus,3608,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_url,426880,413,https://spokane.craigslist.org,2988,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,426880.0,NaN,NaN,NaN,75199.033187,12182282.173598,0.0,5900.0,13950.0,26485.75,3736928711.0
year,425675.0,NaN,NaN,NaN,2011.235191,9.45212,1900.0,2008.0,2013.0,2017.0,2022.0
manufacturer,409234,42,ford,70985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model,421603,29651,f-150,8009,NaN,NaN,NaN,NaN,NaN,NaN,NaN
condition,252776,6,good,121456,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cylinders,249202,8,6 cylinders,94169,NaN,NaN,NaN,NaN,NaN,NaN,NaN


STEP 4 — Drop unnecessary columns



In [8]:
drop_list = ["id", "url", "VIN", "image_url", "description"]  # adjust to your dataset
drop_list = [c for c in drop_list if c in df.columns]
df = df.drop(columns=drop_list)
print("Dropped:", drop_list)

Dropped: ['id', 'url', 'VIN', 'image_url', 'description']


STEP 5 — Fix datatypes (posting_date → datetime, numeric conversions)

In [9]:


# posting_date -> datetime
if "posting_date" in df.columns:
    df["posting_date"] = pd.to_datetime(df["posting_date"], errors="coerce")

# Helper to clean numeric columns that might have commas or dollar signs
def to_numeric_clean(series):
    return pd.to_numeric(series.astype(str).str.replace(r"[^\d\.\-]", "", regex=True), errors="coerce")

# example numeric columns (update to match your CSV)
possible_num_cols = ["price", "odometer", "year", "cylinders"]
num_cols = [c for c in possible_num_cols if c in df.columns]

for c in num_cols:
    df[c] = to_numeric_clean(df[c])

# Trim whitespace from object columns and lower-case if desired
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].astype(str).str.strip()

C:\Users\Prash\AppData\Local\Temp\ipykernel_16208\2561279701.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["posting_date"] = pd.to_datetime(df["posting_date"], errors="coerce")


06 — Quick missing-value summary



In [10]:

missing = df.isnull().sum().sort_values(ascending=False)
missing_percent = (missing / len(df) * 100).round(2)
missing_summary = pd.concat([missing, missing_percent], axis=1)
missing_summary.columns = ["missing_count", "missing_pct"]
missing_summary.head(50)

,missing_count,missing_pct
county,426880,100.00
cylinders,178976,41.93
lat,6549,1.53
long,6549,1.53
odometer,4400,1.03
year,1205,0.28
price,0,0.00
region_url,0,0.00
region,0,0.00
manufacturer,0,0.00


07 — Strategy for missing values (apply thresholds)




In [11]:

# drop columns with > 70% missing (adjust threshold if you want)
high_missing_cols = missing_summary[missing_summary["missing_pct"] > 70].index.tolist()
df = df.drop(columns=high_missing_cols)
print("Dropped high-missing columns:", high_missing_cols)

# For numerical columns, fill median; for categorical, fill 'missing' (example)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

# Impute numeric with median
for c in num_cols:
    median_value = df[c].median()
    df[c] = df[c].fillna(median_value)

# Impute categorical with 'Unknown' or a sensible default
for c in cat_cols:
    df[c] = df[c].replace({"nan": np.nan})  # ensure string 'nan' is treated
    df[c] = df[c].fillna("Unknown")

Dropped high-missing columns: ['county']


STEP 8 — Extract day/month/year from posting_date (if available)

In [12]:
# FINAL FIX for Step 8 — Proper datetime conversion with mixed timezones

# Replace empty strings with NaN
df["posting_date"] = df["posting_date"].replace("", np.nan)

# Force datetime conversion using UTC (fixes mixed timezone issue)
df["posting_date"] = pd.to_datetime(df["posting_date"], errors="coerce", utc=True)

print("After conversion dtype:", df["posting_date"].dtype)

# Now extract date parts ONLY if valid datetime
if pd.api.types.is_datetime64_any_dtype(df["posting_date"]):
    df["posting_day"] = df["posting_date"].dt.day
    df["posting_month"] = df["posting_date"].dt.month
    df["posting_year"] = df["posting_date"].dt.year
    print("Date parts extracted successfully")
else:
    print("posting_date could not be converted — skipping")


After conversion dtype: datetime64[ns, UTC]
Date parts extracted successfully


STEP 9 — Clean Categorical Columns

In [13]:
# STEP 9 — Clean categorical columns

# Clean string values (strip spaces + lowercase)
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip().str.lower()

# Standardize fuel types
if "fuel" in df.columns:
    df["fuel"] = df["fuel"].replace({
        "gasoline": "gas",
        "petrol": "gas",
        "diesel": "diesel",
        "hybrid": "hybrid",
        "other": "other",
        "unknown": "unknown",
        "": "unknown"
    })

# Group rare manufacturers into 'other'
if "manufacturer" in df.columns:
    top20 = df["manufacturer"].value_counts().nlargest(20).index
    df["manufacturer"] = df["manufacturer"].apply(
        lambda x: x if x in top20 else "other"
    )


In [14]:
df["fuel"].value_counts()


fuel
gas         356209
other        30728
diesel       30062
hybrid        5170
unknown       3013
electric      1698
Name: count, dtype: int64

STEP 10 — Remove Outliers (Copy–Paste This Code)

In [15]:

# STEP 10 — Remove Outliers (IQR method)

def remove_outliers_iqr(df_in, col, k=1.5):
    Q1 = df_in[col].quantile(0.25)
    Q3 = df_in[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - k * IQR
    upper = Q3 + k * IQR
    return df_in[(df_in[col] >= lower) & (df_in[col] <= upper)]

# Only apply to columns that exist
cols_to_clean = [col for col in ["price", "odometer"] if col in df.columns]

for col in cols_to_clean:
    before = df.shape[0]
    df = remove_outliers_iqr(df, col)
    after = df.shape[0]
    print(f"Outliers removed in {col}: {before - after}")
    

Outliers removed in price: 8177
Outliers removed in odometer: 4340


STEP 11 — Final check + save cleaned dataset

In [17]:
print("Final shape:", df.shape)
print(df.isnull().sum().sort_values(ascending=False).head(20))

# Save cleaned dataset with a NEW name
df.to_csv("final.csv", index=False)
print("Cleaned dataset saved as cleaned_vehicles_final.csv")


Final shape: (414363, 23)
posting_month    68
posting_day      68
posting_date     68
posting_year     68
region            0
region_url        0
price             0
condition         0
model             0
manufacturer      0
year              0
cylinders         0
fuel              0
odometer          0
title_status      0
type              0
size              0
drive             0
transmission      0
long              0
dtype: int64
Cleaned dataset saved as cleaned_vehicles_final.csv
